# Health Analytics

- **nota:** O conteúdo deste projeto analítico foi feito com apoio de uma pessoa que conhece a áre a da saúde. Portanto é importante que o Analista ou Cientista de dados tenha ajuda dos profissionais que entendem da área ao qual o projeto está direcionado (é bom que o analista ou cientista de dados tenha algum conhecimento sobre o negócio em questão, porém sua funcão é analizar os dados e tirar insights que ajudem na tomada de decisão, ele não tem a obrigação de entender as regras de negócio de cada empreza, por isso é interessante que um profissional da área o acompanhe e forneça informações/auxílio a respeito da área em questão)

**Importando pacotes necessários**

In [4]:
# base requirements
import pandas as pd
import numpy as np


# my package
import datascience.dataset.analysis
Dataset = datascience.dataset.analysis.Dataset

In [5]:
ds = Dataset()